In [7]:
!pip install termcolor
!pip install pandas_ta

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import time
import requests


import warnings
warnings.filterwarnings("ignore")

def access_token(api_secret, request_token, kite):
    user = pd.DataFrame()
    try:
        user = kite.generate_session(request_token, api_secret)
       
        print("Access token ->" + user["access_token"])
        acc_key=user["access_token"]
   
    except Exception as e:
        print("Authentication failed", str(e))
        raise

    print(user["user_name"], "has successfully signed in.")
    return user



acc_key=0

while(acc_key==0):
    user_name = "RK2267"
    password = "Shan@#2021"
#     totp_key = "RGHL2AKPKBTQUPR35WFKWDTIO5NJEALQ"
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disble-dev-shm-usage')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://kite.trade/connect/login?api_key=pv2830q1vbrhu1eu")
    time.sleep(1)
    element = driver.find_element_by_id("userid")
    element.send_keys(user_name)
    element = driver.find_element_by_id("password")
    element.send_keys(password)
    element.send_keys(Keys.RETURN)
    time.sleep(15)
#     outh = pyotp.TOTP(totp_key)
#     element = driver.find_element_by_id('externaltotp')
#     element.send_keys(outh.now())
#     element.send_keys(Keys.RETURN)
#     time.sleep(1)
    # element.close()
    print(driver.current_url)
    url = driver.current_url.split("request_token=")[1].split("&")[0]
    print(url, "new url")
    driver.quit()
    my_api_key = "pv2830q1vbrhu1eu"
    kite = KiteConnect(api_key=my_api_key)
    req_key = url
    user_details = access_token(
        "gz426yatawim9xruj2p6e51c40k44r7n", req_key, kite)
    acc_key = user_details["access_token"]
#     print("Access token ->" + acc_key)
    kite.set_access_token(acc_key)

https://kite.trade/?type=login&status=success&request_token=x6IrslVVqLtfX9MqoL25L5nwEQSqXnro&action=login
x6IrslVVqLtfX9MqoL25L5nwEQSqXnro new url
Access token ->Bnmmlp2PbTwgwU2CMkPGz8hj5aYb0xhW
Kakal Krishna Rao has successfully signed in.


In [86]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))

In [87]:
#%%capture cap --no-stderr

from termcolor import colored
import pandas_ta as ta
from datetime import *

def history_data(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now() - timedelta(days=100))
    curr_date=datetime.now()
    if 9<=curr_date.hour<16:
        new_date=(datetime.now() - timedelta(days=1))
    else:
        new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10]+" "+'00:00:00.0')
    endd=pd.to_datetime(str(new_date)[:10]+' '+'15:30:00.0')
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1


def crossover(dfx,tkname):
    def EMA(df, base, target, period, alpha=False):
        con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
        if (alpha == True):
            df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
        else:
            df[target] = con.ewm(span=period, adjust=False).mean()

        df[target].fillna(0, inplace=True)
        return df

    def ATR(df, period, ohlc=['open', 'high', 'low', 'close']):
        atr = 'ATR_' + str(period)

        if not 'TR' in df.columns:
            df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
            df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
            df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())

            df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)

            df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)

        EMA(df, 'TR', atr, period, alpha=True)

        return df

    def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):

        ATR(df, period, ohlc=ohlc)
        atr = 'ATR_' + str(period)
        st = 'ST' #+ str(period) + '_' + str(multiplier)
        stx = 'STX' #  + str(period) + '_' + str(multiplier
        df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
        df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]
        df['final_ub'] = 0.00
        df['final_lb'] = 0.00
        for i in range(period, len(df)):
            df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or \
                                                             df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else \
            df['final_ub'].iat[i - 1]
            df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or \
                                                             df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else \
            df['final_lb'].iat[i - 1]

        df[st] = 0.00
        for i in range(period, len(df)):
            df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[
                i] <= df['final_ub'].iat[i] else \
                df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] > \
                                         df['final_ub'].iat[i] else \
                    df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= \
                                             df['final_lb'].iat[i] else \
                        df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] < \
                                                 df['final_lb'].iat[i] else 0.00

        df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down', 'up'), np.NaN)
        df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb'], inplace=True, axis=1)

        df.fillna(0, inplace=True)
        return 
    def macd(df,m1,m2,s1):
        exp1 = df.Close.ewm(span=m1, adjust=False).mean()
        exp2 = df.Close.ewm(span=m2, adjust=False).mean()
        macd2 = exp1-exp2
        exp3 = macd2.ewm(span=s1, adjust=False).mean()
        return macd2,exp3
    def change(s):
        return s[:10]

    dfx['Time']=dfx.Time.apply(change)
    macd,sig=macd(dfx,12,26,9)
    dfx['macd']=macd
    dfx['sig']=sig
    SuperTrend(dfx,7,1)
    dfx['RSI']=ta.rsi(dfx['Close'],length=14)
    
    last10=dfx.loc[len(dfx)-11:]
    buy=0
    sell=0
    nocross=0
    print('LAST 10DAYS DETAILS OF',colored(tkname,'blue'))
    last10.reset_index(inplace=True,drop=True)
    print()
    for i in range(1,len(last10)):
        print(colored('Date:','magenta'),last10['Time'].iloc[i],colored('Close:','magenta'),last10['Close'].iloc[i],colored('MACD:','magenta'),round(last10['macd'].iloc[i],3),colored('SIG:','magenta'),round(last10['sig'].iloc[i],3),colored('ST:','magenta'),round(last10['ST'].iloc[i],3),colored('RSI:','magenta'),round(last10['RSI'].iloc[i],3))
    print()
    print("\u0332".join('DATE      MACD  ST   RSI '))
    print()
    bothstrsi=0
    bothmacdrsi=0
    bothmacst=0
    macsts=0
    macdrsis=0
    strsis=0
    macl=0
    macs=0
    stl=0
    sts=0
    rsil=0
    rsis=0
    total=0
    total2=0
    
    for i in range(1,len(last10)):
        s=""
        result=[-1]*3
        if last10['RSI'].iloc[i]>55 and last10['RSI'].iloc[i-1]<50:
            result[2]=1
        elif last10['RSI'].iloc[i]<55 and last10['RSI'].iloc[i-1]>50:
            result[2]=0
            
        if last10['Close'].iloc[i]>last10['ST'].iloc[i] and last10['Close'].iloc[i-1]<last10['ST'].iloc[i-1]:
            result[1]=1
        elif last10['Close'].iloc[i]<last10['ST'].iloc[i] and last10['Close'].iloc[i-1]>last10['ST'].iloc[i-1]:
            result[1]=0
        
        if last10['macd'].iloc[i]>last10['sig'].iloc[i] and last10['macd'].iloc[i-1]<last10['sig'].iloc[i-1]:
            result[0]=1
        elif last10['macd'].iloc[i]<last10['sig'].iloc[i] and last10['macd'].iloc[i-1]>last10['sig'].iloc[i-1]:
            result[0]=0
        
        if result[0]==-1:
            s+='NA'+'    '
        elif result[0]==1:
            s+=colored('LONG','green')+"  "
            macl+=1
        elif result[0]==0:
            s+=colored('SHORT','red')+"  "
            macs+=1
        
        if result[1]==-1:
            s+='NA'+'    '
        elif result[1]==1:
            s+=colored('LONG','green')+"  "
            stl+=1
        elif result[1]==0:
            s+=colored('SHORT','red')+"  "
            sts+=1
            
        if result[2]==-1:
            s+='NA'+'    '
        elif result[2]==1:
            s+=colored('LONG','green')+"  "
            rsil+=1
        elif result[2]==0:
            s+=colored('SHORT','red')+"  "
            rsis+=1
            
        if result[0]+result[1]==2:
            bothmacst+=1
        if result[1]+result[2]==2:
            bothstrsi+=1
        if result[0]+result[2]==2:
            bothmacdrsi+=1
        if result[0]+result[1]+result[2]==3:
            total+=1
            
        if result[0]==0 and result[1]==0:
            macsts+=1
        if result[0]==0 and result[2]==0:
            macdrsis+=1
        if result[1]==0 and result[2]==0:
            strsis+=1
        if result[0]==0 and result[1]==0 and result[2]==0:
            total2+=1         
        print(last10['Time'].iloc[i],s)      
    print()
    print('------ ANALYSIS PART ---------')
    
    print('MACD LONG :',colored(macl,'green') if macl>0 else macl)
    print('MACD SHORT :',colored(macs,'red') if macs>0 else macs)
    print('ST LONG :',colored(stl,'green') if stl>0 else stl)
    print('ST SHORT :',colored(sts,'red') if sts>0 else sts)
    print('RSI LONG :',colored(rsil,'green') if rsil>0 else rsil)
    print('RSI SHORT :',colored(rsis,'red') if rsis>0 else rsis)
    print('BOTH MACD AND ST GAVE LONG :',colored(bothmacst,'green') if bothmacst>0 else bothmacst )
    print('BOTH ST AND RSI GAVE LONG :',colored(bothstrsi,'green') if bothstrsi>0 else bothstrsi )
    print('BOTH MACD AND RSI GAVE LONG :',colored(bothmacdrsi,'green')if bothmacdrsi>0 else bothmacdrsi )
    print('MACD,ST AND RSI ALL GAVE LONG :',colored(total,'green') if total>0 else total)
    print('BOTH MACD AND ST GAVE SHORT :',colored(macsts,'red') if macsts>0 else macsts )
    print('BOTH ST AND RSI GAVE SHORT :',colored(strsis,'red') if strsis>0 else strsis)
    print('BOTH MACD AND RSI GAVE SHORT :',colored(macdrsis,'red')if macdrsis>0 else macdrsis )
    print('MACD,ST AND RSI ALL GAVE SHORT :',colored(total2,'red') if total2>0 else total2)
    print()
    print('------------------------------------------------------------------------------------------------------------')

dicct={67329: 'ATUL', 5436929: 'AUBANK', 70401: 'AUROPHARMA', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 85761: 'BALKRISIND', 87297: 'BALRAMCHIN', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 94977: 'BATAINDIA', 98049: 'BEL', 103425: 'BERGEPAINT', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2911489: 'BIOCON', 558337: 'BOSCHLTD', 134657: 'BPCL'}
tokens=[67329, 5436929, 70401, 1510401, 4267265, 4268801, 81153, 85761, 87297, 579329, 1195009, 94977, 98049, 103425, 108033, 2714625, 112129, 2911489, 558337, 134657]
for tk in tokens:
    df=history_data(tk)[0]
    crossover(df,dicct[tk])

LAST 10DAYS DETAILS OF ATUL

Date: 2022-09-30 Close: 9023.9 MACD: -47.93 SIG: 17.447 ST: 9063.925 RSI: 45.364
Date: 2022-10-03 Close: 8919.6 MACD: -59.35 SIG: 2.088 ST: 9063.925 RSI: 42.212
Date: 2022-10-04 Close: 9039.55 MACD: -58.051 SIG: -9.94 ST: 9063.925 RSI: 46.791
Date: 2022-10-06 Close: 9001.7 MACD: -59.392 SIG: -19.831 ST: 9063.925 RSI: 45.564
Date: 2022-10-07 Close: 8900.4 MACD: -67.846 SIG: -29.434 ST: 9063.925 RSI: 42.363
Date: 2022-10-10 Close: 8767.95 MACD: -84.262 SIG: -40.399 ST: 9032.226 RSI: 38.549
Date: 2022-10-11 Close: 8671.85 MACD: -103.83 SIG: -53.086 ST: 8911.458 RSI: 36.015
Date: 2022-10-12 Close: 8696.5 MACD: -116.011 SIG: -65.671 ST: 8870.078 RSI: 37.156
Date: 2022-10-13 Close: 8763.2 MACD: -118.912 SIG: -76.319 ST: 8870.078 RSI: 40.26
Date: 2022-10-14 Close: 8764.75 MACD: -119.706 SIG: -84.996 ST: 8870.078 RSI: 40.334

D̲A̲T̲E̲ ̲ ̲ ̲ ̲ ̲ ̲M̲A̲C̲D̲ ̲ ̲S̲T̲ ̲ ̲ ̲R̲S̲I̲ 

2022-09-30 NA    NA    NA    
2022-10-03 NA    NA    NA    
2022-10-04 NA    NA    NA    


LAST 10DAYS DETAILS OF BAJAJ-AUTO

Date: 2022-09-30 Close: 3527.75 MACD: -114.622 SIG: -89.491 ST: 3596.676 RSI: 26.613
Date: 2022-10-03 Close: 3515.35 MACD: -116.027 SIG: -94.798 ST: 3596.676 RSI: 25.963
Date: 2022-10-04 Close: 3579.6 MACD: -110.681 SIG: -97.975 ST: 3596.676 RSI: 34.845
Date: 2022-10-06 Close: 3594.65 MACD: -104.03 SIG: -99.186 ST: 3596.676 RSI: 36.759
Date: 2022-10-07 Close: 3603.55 MACD: -96.924 SIG: -98.733 ST: 3541.384 RSI: 37.92
Date: 2022-10-10 Close: 3616.95 MACD: -89.183 SIG: -96.823 ST: 3541.384 RSI: 39.715
Date: 2022-10-11 Close: 3528.75 MACD: -89.138 SIG: -95.286 ST: 3661.868 RSI: 32.96
Date: 2022-10-12 Close: 3624.5 MACD: -80.448 SIG: -92.319 ST: 3661.868 RSI: 44.08
Date: 2022-10-13 Close: 3604.8 MACD: -74.295 SIG: -88.714 ST: 3661.868 RSI: 42.517
Date: 2022-10-14 Close: 3570.5 MACD: -71.364 SIG: -85.244 ST: 3661.868 RSI: 39.867

D̲A̲T̲E̲ ̲ ̲ ̲ ̲ ̲ ̲M̲A̲C̲D̲ ̲ ̲S̲T̲ ̲ ̲ ̲R̲S̲I̲ 

2022-09-30 NA    NA    NA    
2022-10-03 NA    NA    NA    
2022-10-04 NA    

LAST 10DAYS DETAILS OF BALRAMCHIN

Date: 2022-09-30 Close: 347.7 MACD: -3.902 SIG: -0.527 ST: 355.589 RSI: 43.518
Date: 2022-10-03 Close: 344.7 MACD: -4.388 SIG: -1.299 ST: 355.589 RSI: 41.991
Date: 2022-10-04 Close: 348.7 MACD: -4.399 SIG: -1.919 ST: 355.589 RSI: 44.773
Date: 2022-10-06 Close: 360.4 MACD: -3.424 SIG: -2.22 ST: 345.319 RSI: 52.019
Date: 2022-10-07 Close: 362.95 MACD: -2.419 SIG: -2.26 ST: 350.881 RSI: 53.453
Date: 2022-10-10 Close: 357.9 MACD: -2.006 SIG: -2.209 ST: 350.881 RSI: 50.251
Date: 2022-10-11 Close: 351.6 MACD: -2.162 SIG: -2.2 ST: 350.881 RSI: 46.508
Date: 2022-10-12 Close: 358.85 MACD: -1.681 SIG: -2.096 ST: 350.881 RSI: 51.028
Date: 2022-10-13 Close: 353.9 MACD: -1.68 SIG: -2.013 ST: 350.881 RSI: 48.043
Date: 2022-10-14 Close: 349.5 MACD: -2.012 SIG: -2.013 ST: 365.891 RSI: 45.495

D̲A̲T̲E̲ ̲ ̲ ̲ ̲ ̲ ̲M̲A̲C̲D̲ ̲ ̲S̲T̲ ̲ ̲ ̲R̲S̲I̲ 

2022-09-30 NA    NA    NA    
2022-10-03 NA    NA    NA    
2022-10-04 NA    NA    NA    
2022-10-06 NA    LONG  NA    
2022-1

LAST 10DAYS DETAILS OF BEL

Date: 2022-09-30 Close: 100.95 MACD: -0.073 SIG: 1.814 ST: 102.545 RSI: 42.508
Date: 2022-10-03 Close: 98.45 MACD: -0.498 SIG: 1.352 ST: 102.545 RSI: 37.569
Date: 2022-10-04 Close: 102.05 MACD: -0.539 SIG: 0.973 ST: 102.545 RSI: 47.1
Date: 2022-10-06 Close: 104.65 MACD: -0.357 SIG: 0.707 ST: 99.916 RSI: 52.715
Date: 2022-10-07 Close: 106.25 MACD: -0.083 SIG: 0.549 ST: 101.65 RSI: 55.823
Date: 2022-10-10 Close: 104.45 MACD: -0.011 SIG: 0.437 ST: 101.65 RSI: 51.706
Date: 2022-10-11 Close: 103.05 MACD: -0.066 SIG: 0.337 ST: 101.65 RSI: 48.698
Date: 2022-10-12 Close: 103.25 MACD: -0.092 SIG: 0.251 ST: 101.65 RSI: 49.153
Date: 2022-10-13 Close: 101.4 MACD: -0.259 SIG: 0.149 ST: 105.433 RSI: 45.162
Date: 2022-10-14 Close: 100.8 MACD: -0.435 SIG: 0.032 ST: 105.242 RSI: 43.916

D̲A̲T̲E̲ ̲ ̲ ̲ ̲ ̲ ̲M̲A̲C̲D̲ ̲ ̲S̲T̲ ̲ ̲ ̲R̲S̲I̲ 

2022-09-30 NA    NA    NA    
2022-10-03 NA    NA    NA    
2022-10-04 NA    NA    NA    
2022-10-06 NA    LONG  NA    
2022-10-07 NA    NA 

LAST 10DAYS DETAILS OF BHEL

Date: 2022-09-30 Close: 59.9 MACD: 0.241 SIG: 0.758 ST: 57.192 RSI: 55.345
Date: 2022-10-03 Close: 60.55 MACD: 0.371 SIG: 0.68 ST: 58.815 RSI: 57.309
Date: 2022-10-04 Close: 62.0 MACD: 0.585 SIG: 0.661 ST: 59.741 RSI: 61.388
Date: 2022-10-06 Close: 63.85 MACD: 0.893 SIG: 0.707 ST: 61.296 RSI: 65.869
Date: 2022-10-07 Close: 63.9 MACD: 1.128 SIG: 0.792 ST: 61.529 RSI: 65.984
Date: 2022-10-10 Close: 63.2 MACD: 1.244 SIG: 0.882 ST: 61.529 RSI: 62.797
Date: 2022-10-11 Close: 60.95 MACD: 1.141 SIG: 0.934 ST: 64.23 RSI: 53.801
Date: 2022-10-12 Close: 61.6 MACD: 1.099 SIG: 0.967 ST: 63.001 RSI: 55.772
Date: 2022-10-13 Close: 60.65 MACD: 0.978 SIG: 0.969 ST: 63.001 RSI: 52.262
Date: 2022-10-14 Close: 61.55 MACD: 0.943 SIG: 0.964 ST: 63.001 RSI: 55.142

D̲A̲T̲E̲ ̲ ̲ ̲ ̲ ̲ ̲M̲A̲C̲D̲ ̲ ̲S̲T̲ ̲ ̲ ̲R̲S̲I̲ 

2022-09-30 NA    NA    NA    
2022-10-03 NA    NA    NA    
2022-10-04 NA    NA    NA    
2022-10-06 LONG  NA    NA    
2022-10-07 NA    NA    NA    
2022-10-10 NA   